# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chapaev,50.2000,51.1667,14.72,75,5,3.13,KZ,1722042518
1,1,quepos,9.4236,-84.1652,25.88,88,99,1.25,CR,1722042519
2,2,constantia,44.1833,28.6500,18.38,81,0,5.28,RO,1722042521
3,3,vorgashor,67.5833,63.9500,8.60,94,100,2.19,RU,1722042522
4,4,howrah,22.5892,88.3103,29.00,94,75,2.06,IN,1722042523


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df=city_data_df[
    (city_data_df["Max Temp"]>24) & (city_data_df["Max Temp"]<28)\
    & (city_data_df["Wind Speed"]<5)\
    & (city_data_df["Humidity"]<50)\
    & (city_data_df["Cloudiness"]==0)
]

# Drop any rows with null values
ideal_city_df=ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
281,281,tataouine,32.9297,10.4518,25.71,49,0,1.67,TN,1722042871
350,350,tamanrasset,22.7850,5.5228,24.95,19,0,1.03,DZ,1722042957
377,377,lesina,41.8594,15.3527,24.30,42,0,1.16,IT,1722042991
404,404,moires,35.0532,24.8744,26.79,47,0,2.26,GR,1722043025
541,541,buka,40.8108,69.1986,24.43,33,0,1.74,UZ,1722043200
571,571,ouro preto do oeste,-10.7481,-62.2158,24.49,43,0,1.06,BR,1722043236


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
281,tataouine,TN,32.9297,10.4518,49,
350,tamanrasset,DZ,22.7850,5.5228,19,
377,lesina,IT,41.8594,15.3527,42,
404,moires,GR,35.0532,24.8744,47,
541,buka,UZ,40.8108,69.1986,33,
571,ouro preto do oeste,BR,-10.7481,-62.2158,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address =  name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tataouine - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
lesina - nearest hotel: Palazzo Ducale residence Liù
moires - nearest hotel: Hotel Olympic
buka - nearest hotel: No hotel found
ouro preto do oeste - nearest hotel: Graúna Resort Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
281,tataouine,TN,32.9297,10.4518,49,No hotel found
350,tamanrasset,DZ,22.7850,5.5228,19,فندق الأمان
377,lesina,IT,41.8594,15.3527,42,Palazzo Ducale residence Liù
404,moires,GR,35.0532,24.8744,47,Hotel Olympic
541,buka,UZ,40.8108,69.1986,33,No hotel found
571,ouro preto do oeste,BR,-10.7481,-62.2158,43,Graúna Resort Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name","Country"]
)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)